In [ ]:
import talib
import MetaTrader5 as mt  
import pandas as pd  
import time
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model

mt.initialize()
login = 51225190
password = 'W5bu8LY1V'
server = 'Alpari-MT5-Demo'

mt.login(login, password, server)
# get account info
account_info = mt.account_info()
print(account_info)

# getting specific account data
login_number = account_info.login
balance = account_info.balance
equity = account_info.equity


model = load_model('C:/Users/vovad/Desktop/TRADING/FOR_TRADE_1.h5')

def dataset_import():
    dataset1 = mt.copy_rates_from_pos("EURUSD_i", mt.TIMEFRAME_H1, 0, 1000)    
    dataset1 = pd.DataFrame(dataset1)        
    dataset = dataset1.drop(['time', 'spread', 'real_volume'], axis=1)        
    open = dataset['open']
    high = dataset['high']
    low = dataset['low']
    close = dataset['close']
    dataset['EMA_24'] = talib.EMA(close, timeperiod=24)
    dataset['EMA_60'] = talib.EMA(close, timeperiod=60)
    dataset['EMA_120'] = talib.EMA(close, timeperiod=120)
    dataset['upperband'], dataset['middleband'], dataset['lowerband'] = talib.BBANDS(close, timeperiod=24, nbdevup=8, nbdevdn=8, matype=0)
    dataset['HT_TRENDLINE'] = talib.HT_TRENDLINE(close)
    dataset['TEMA'] = talib.TEMA(close, timeperiod=24)
    dataset['AROONOSC'] = talib.AROONOSC(high, low, timeperiod=24)
    dataset['macd'], dataset['macdsignal'], dataset['macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=24, signalperiod=9)
    dataset['RSI'] = talib.RSI(close, timeperiod=24)
    dataset['HT_DCPERIOD'] = talib.HT_DCPERIOD(close)
    dataset['HT_DCPHASE'] = talib.HT_DCPHASE(close)
    dataset['HT_TRENDMODE'] = talib.HT_TRENDMODE(close)
    dataset['NATR'] = talib.NATR(high, low, close, timeperiod=24)
    dataset = dataset.dropna()
    dataset = dataset[['open', 'high', 'low', 'close', 'tick_volume', 'EMA_24', 'EMA_60', 'EMA_120', 'upperband', 'middleband', 'lowerband', 'HT_TRENDLINE', 'TEMA', 'AROONOSC', 'macd', 'macdsignal', 'macdhist', 
                 'RSI', 'HT_DCPERIOD', 'HT_DCPHASE', 'HT_TRENDMODE', 'NATR']]
    dataset = dataset.values               
    scaler = MinMaxScaler()
    x_test = scaler.fit_transform(dataset)
    preds = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
    preds_new = model.predict(preds, verbose=0)
    preds = np.argmax(preds_new, axis=-1)
    return preds

def order_buy(symbol, volume, **kwargs):
      
    point = mt.symbol_info(symbol).point
    price = mt.symbol_info_tick(symbol).ask
    deviation = 20
    request = {
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": volume,
    "type": mt.ORDER_TYPE_BUY,
    "price": price,
    "sl": price - 150 * point,
    "tp": price + 300 * point,
    "deviation": DEVIATION,
    "magic": 234000,
    "comment": "python script open",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_RETURN,
    }   

    order_result = mt.order_send(request)
    print(order_result)

    return order_result

def order_sell(symbol, volume, **kwargs):
      
    point = mt.symbol_info(symbol).point
    price = mt.symbol_info_tick(symbol).bid
    deviation = 20
    request = {
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": volume,
    "type": mt.ORDER_TYPE_SELL,
    "price": price,
    "sl": price + 150 * point,
    "tp": price - 300 * point,
    "deviation": DEVIATION,
    "magic": 234000,
    "comment": "python script open",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_RETURN,
    }   

    order_result = mt.order_send(request)
    print(order_result)

    return order_result
    
def close_order_sell(ticket, symbol):
    positions = mt.positions_get()

    for pos in positions:        

        if pos.ticket == ticket:
            price = mt.symbol_info_tick(symbol).bid
            
            request={
            "action": mt.TRADE_ACTION_DEAL,
            "symbol": pos.symbol,
            "volume": pos.volume,
            "type": mt.ORDER_TYPE_SELL,
            "position": pos.ticket,
            "price": price,
            "deviation": DEVIATION,
            "magic": 234000,
            "comment": "python script close",
            "type_time": mt.ORDER_TIME_GTC,
            "type_filling": mt.ORDER_FILLING_RETURN,
            }

            order_result = mt.order_send(request)
            print(order_result)

            return order_result

    return 'Ticket does not exist'

def close_order_buy(ticket, symbol):
    positions = mt.positions_get()

    for pos in positions:        

        if pos.ticket == ticket:
            price = mt.symbol_info_tick(symbol).ask
            
            request={
            "action": mt.TRADE_ACTION_DEAL,
            "symbol": pos.symbol,
            "volume": pos.volume,
            "type": mt.ORDER_TYPE_BUY,
            "position": pos.ticket,
            "price": price,
            "deviation": DEVIATION,
            "magic": 234000,
            "comment": "python script close",
            "type_time": mt.ORDER_TIME_GTC,
            "type_filling": mt.ORDER_FILLING_RETURN,
            }

            order_result = mt.order_send(request)
            print(order_result)

            return order_result

    return 'Ticket does not exist'

def signal(df):
    signal = df[-1] 

    direction = 'flat'
    if signal == 1:
        direction = 'buy'
    elif signal == 0:
        direction = 'sell'
    elif signal == 2:
        pass

    return direction

# function to trail SL
def trail_sl():
    # get position based on ticket_id
    position = mt.positions_get()

    # check if position exists
    if position:
        position = position[0]
    else:
        print('Position does not exist')
        # sys.exit()

    # get position data
    order_type = position.type
    price_current = position.price_current
    price_open = position.price_open
    sl = position.sl

    dist_from_sl = abs(round(price_current - sl, 6))

    if dist_from_sl > MAX_DIST_SL:
        # calculating new sl
        if sl != 0.0:
            if order_type == 0:  # 0 stands for BUY
                new_sl = sl + TRAIL_AMOUNT

            elif order_type == 1:  # 1 stands for SELL
                new_sl = sl - TRAIL_AMOUNT

        else:
            # setting default SL if the is no SL on the symbol
            new_sl = price_open - DEFAULT_SL if order_type == 0 else price_open + DEFAULT_SL

        request = {
            'action': mt.TRADE_ACTION_SLTP,
            'position': position.ticket,
            'sl': new_sl,
        }

        result = mt.order_send(request)
        print(result)
        return result

if __name__ == '__main__':

    # strategy parameters
    MAX_DIST_SL = 0.004  # Max distance between current price and SL, otherwise SL will update
    TRAIL_AMOUNT = 0.00003  # Amount by how much SL updates
    DEFAULT_SL = 0.004  # If position has no SL, set a default SL

    SYMBOL = "EURUSD_i"
    VOLUME = 0.01
    TIMEFRAME = mt.TIMEFRAME_M15
    DEVIATION = 20

    mt.initialize()
    
    while True:
        dataset = dataset_import()        
        direction = signal(dataset)
        

        if direction == 'buy':
            
            for pos in mt.positions_get():
                if pos.type == 1:
                    close_order_buy(pos.ticket, SYMBOL)

            # if there are no open positions, open a new long position
            if not mt.positions_total():
                order_buy(SYMBOL, VOLUME)
                result = trail_sl()

        elif direction == 'sell':
            # if we have a SELL signal, close all short positions
            for pos in mt.positions_get():
                if pos.type == 0:  # pos.type == 1 represent a buy order
                    close_order_sell(pos.ticket, SYMBOL)

            # if there are no open positions, open a new short position
            if not mt.positions_total():
                order_sell(SYMBOL, VOLUME)
                result = trail_sl()
        


        # time.sleep(5)
        